In [1]:
import numpy 
numpy.random.seed(1337)

In [2]:
import h5py
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, MaxPooling2D, Dropout
from keras.layers import Conv2D, Embedding
from keras.optimizers import SGD, rmsprop
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support, accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from keras.layers.advanced_activations import LeakyReLU, PReLU, ELU

Using TensorFlow backend.


In [3]:
batch_size=28 #128
#embedding_dims=3
num_classes=37 #37 #1227 #40499 #4
epochs=22

In [4]:
import os
from PIL import Image
import matplotlib.image as mpimg

In [5]:


# num_images=100
# dim=(100,100,1)
# x=numpy.zeros((num_images, 100,100,1))
# for fol in classes:
#     imgfiles=os.listdir(path+u'\\'+fol);
#     for i, img in enumerate(imgfiles):
#         im=Image.open(path+'\\'+fol+'\\'+img)
#         x[i]=numpy.asarray(im)/255
    
# print (x.shape)


os.chdir("E:");
path="E:/Dummy/NN2";
# path="newdataset/1";
# path="newdataset/2";
# path="Dataset-3characters-only - CC -DC";
# path="Dataset_3_MoreFonts/1";
classes=os.listdir(path)
x=[]#Datapoints 830
y=[]#labels 3
for fol in classes:
    #print (fol)
    imgfiles=os.listdir(path+u'\\'+fol);
    #print (imgfiles)
    for img in imgfiles:
       # print (img)
        #im=Image.open(path+u'\\'+fol+u'\\'+img);
        im=mpimg.imread(path+u'\\'+fol+u'\\'+img);
        #im.show()
        #im=numpy.asarray(im)/255;
       # print(im)
        x.append(im)
        y.append(fol)
x=numpy.array(x)
#print (x)
y=numpy.array(y)
#print(y)
# numpy.savez('C:/Dataset_Final(March)/xlevel0',x)
# numpy.savez('C:/Dataset_Final(March)/ylevel0',y)

In [6]:
# x=numpy.load('C:/Dataset_Final(March)/xlevel0.npz')
# x=x['arr_0']
# print (x.shape)
# y=numpy.load('C:/Dataset_Final(March)/xlevel0.npz')
# y=y['arr_0']
# print (y.shape)


Using HDF5 

In [7]:
print(x.shape)
print(y.shape)
#x=keras.utils.HDF5Matrix('Dataset_Final(March)/One.hdf5',x)
#model.predict(x)
x=x.reshape((-1,100,100,1))
#print (x)

(144, 100, 100)
(144,)


In [8]:
n=x.shape[0]
randomize=numpy.arange(n)
numpy.random.shuffle(randomize)
randomize
x=x[randomize]
y=y[randomize]

In [9]:
test_split=round(n*2/3)
x_train=x[:test_split]
y_train=y[:test_split]
x_test=x[test_split:]
y_test=y[test_split:]

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
#y_train[:4]


#print (class_weight)
#print (class_weight_list)

(96, 100, 100, 1)
(96,)
(48, 100, 100, 1)
(48,)


In [10]:

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
 featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

In [11]:
#x_train=x_train.reshape((-1,100,100,1))
#x_test=x_test.reshape(-1,100,100,1)

x_train=x_train.astype('float32')
x_test=x_test.astype('float32')

x_train/=255
x_test/=255
#plt.imshow(x_train[0])

print(x_train.shape[0])
print(x_test.shape[0])
print(x_train.shape)
print(x_test.shape)

96
48
(96, 100, 100, 1)
(48, 100, 100, 1)


In [12]:
# encoder = LabelEncoder()
# encoder.fit(y_train)
# y_train= encoder.transform(y_train)
# y_test= encoder.transform(y_test)

# class_weight_list = compute_class_weight('balanced', numpy.unique(y_train), y_train)
# class_weight = dict(zip(numpy.unique(y_train), class_weight_list))
# print (class_weight)

y_train=keras.utils.to_categorical(y_train, num_classes)
y_test=keras.utils.to_categorical(y_test, num_classes)
y_train[:4]  
#print (y_train.shape)

#class_weight = class_weight.compute_class_weight('balanced', numpy.unique(y_train), y_train)

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [34]:
inputs=Input(shape=(100,100,1))

# x=Conv2D(32,(32,32), padding='same')(inputs)
# x=LeakyReLU(alpha=0.1)(x) 
# #x=PReLU(alpha_initializer="zero")(x) 
# #x=Activation('Relu')(x)
# #x=ELU(alpha=1.0)(x) 
# x=Conv2D(16,(12,12))(x)
# x=LeakyReLU(alpha=0.1)(x)
# #x=PReLU(alpha_initializer="zero")(x) 
# #x=ELU(alpha=1.0)(x) 
# x=Conv2D(8,(3,3))(x)
# x=LeakyReLU(alpha=0.1)(x)
# #x=PReLU(alpha_initializer="zero")(x) 
# #x=Activation('Relu')(x)
# #x=ELU(alpha=1.0)(x) 
# x=MaxPooling2D(pool_size=(2,2))(x)
# x=Dropout(0.2)(x)
# x=Flatten()(x)
# x=Dense(num_classes)(x)
# #x=Activation('softmax')(x)
# output=Activation('softmax')(x)
# model=Model([inputs], output)

In [83]:
x=Conv2D(32, (3, 3), padding='same')(inputs)
x=LeakyReLU(alpha=0.1)(x)
x=MaxPooling2D(pool_size=(2,2))(x)
x=Conv2D(8, (3, 3), padding='same')(x)
x=LeakyReLU(alpha=0.1)(x)
x=MaxPooling2D(pool_size=(2,2))(x)
x=Flatten()(x)
x=Dense(num_classes)(x)
x=LeakyReLU(alpha=0.1)(x)
x=Dropout(0.2)(x)
x=Dense(num_classes)(x)
output=Activation('softmax')(x)
model=Model([inputs], output)

In [69]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',input_shape=(100, 100, 1)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(6, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(6, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(num_classes))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [56]:
# inputs=Input(shape=(100,100,1))

# x=Conv2D(16,(3,3), padding='same')(inputs)
# x=Activation('relu')(x)
# x=Conv2D(8,(3,3))(x)
# x=Activation('relu')(x)
# x=MaxPooling2D(pool_size=(2,2))(x)
# x=Dropout(0.2)(x)
# x=Flatten()(x)
# x=Dense(num_classes)(x)
# #x=Activation('softmax')(x)
# output=Activation('softmax')(x)
# model=Model([inputs], output)

In [84]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100, 100, 1)       0         
_________________________________________________________________
conv2d_61 (Conv2D)           (None, 100, 100, 32)      320       
_________________________________________________________________
leaky_re_lu_25 (LeakyReLU)   (None, 100, 100, 32)      0         
_________________________________________________________________
max_pooling2d_43 (MaxPooling (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 50, 50, 8)         2312      
_________________________________________________________________
leaky_re_lu_26 (LeakyReLU)   (None, 50, 50, 8)         0         
_________________________________________________________________
max_pooling2d_44 (MaxPooling (None, 25, 25, 8)         0         
__________

In [85]:
opt=keras.optimizers.rmsprop(lr=0.0000001,decay=1e-6 )
#opt=keras.optimizers.Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#opt=keras.optimizers.Adamax(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [86]:
import h5py
from keras.callbacks import ModelCheckpoint
filepath='E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_22/weights-improvement-{epoch:2d}-{val_acc:.2f}.hdf5'
checkpoint=ModelCheckpoint(filepath, monitor='val_acc', verbose=1, mode='max')
hist=model.fit(x_train, y_train, 
               batch_size=batch_size,
               epochs=epochs,
               validation_data=(x_test, y_test),
               callbacks=[checkpoint]
               #class_weight=class_weight
               )


Train on 96 samples, validate on 48 samples
Epoch 1/22
96/96 [==============================] - 3s 29ms/step - loss: 3.6109 - acc: 0.0313 - val_loss: 3.6109 - val_acc: 0.0833

Epoch 00001: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_22/weights-improvement- 1-0.08.hdf5
Epoch 2/22
96/96 [==============================] - 2s 19ms/step - loss: 3.6109 - acc: 0.0208 - val_loss: 3.6109 - val_acc: 0.0833

Epoch 00002: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_22/weights-improvement- 2-0.08.hdf5
Epoch 3/22
96/96 [==============================] - 2s 19ms/step - loss: 3.6109 - acc: 0.0313 - val_loss: 3.6109 - val_acc: 0.0833

Epoch 00003: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_22/weights-improvement- 3-0.08.hdf5
Epoch 4/22
96/96 [==============================] - 2s 21ms/step - loss: 3.6109 - acc: 0.0208 - val_loss: 3.6109 - val_acc: 0.0833

Epoch 00004: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_22/weigh

In [ ]:
# for e in range(epochs):
#     print('Epoch', e)
#     batches = 0
#     for x_batch, y_batch in datagen.flow(x_train, y_train, batch_size=32):
#         model.fit(x_batch, y_batch)
#         batches += 1
#         if batches >= len(x_train) / 32:
#             # we need to break the loop by hand because
#             # the generator loops indefinitely
#             break
# hist=model.fit_generator(datagen.flow(x_train, y_train,
#                batch_size=batch_size),
#                epochs=epochs,
#                validation_data=(x_test, y_test),
#                callbacks=[checkpoint]
#                #workers=4
#                #class_weight=class_weight
#                )
# datagen = ImageDataGenerator(
#         featurewise_center=False,  # set input mean to 0 over the dataset
#         samplewise_center=False,  # set each sample mean to 0
#         featurewise_std_normalization=False,  # divide inputs by std of the dataset
#         samplewise_std_normalization=False,  # divide each input by its std
#         zca_whitening=False,  # apply ZCA whitening
#         rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
#         width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
#         height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
#         horizontal_flip=True,  # randomly flip images
#         vertical_flip=False)  # randomly flip images

#     # Compute quantities required for feature-wise normalization
#     # (std, mean, and principal components if ZCA whitening is applied).
# datagen.fit(x_train)

#     # Fit the model on the batches generated by datagen.flow().
# model.fit_generator(datagen.flow(x_train, y_train,
#                                      batch_size=batch_size),
#                         epochs=epochs,
#                         callbacks=[checkpoint],
#                         validation_data=(x_test, y_test),
#                         workers=4)

In [327]:
score=model.evaluate(x_test, y_test)

48/48 [==============================] - 0s 4ms/step


In [328]:
score[0]

3.6109507878621421

In [46]:
score[1]

0.0

In [102]:
model.load_weights("E:/Dataset_Final(March)/Checkpoints/Level_1/weights/weights-improvement- 1-0.00.hdf5")

ValueError: Dimension 0 in both shapes must be equal, but are 3 and 64. Shapes are [3,3,1,32] and [64,1,3,3]. for 'Assign' (op: 'Assign') with input shapes: [3,3,1,32], [64,1,3,3].

In [16]:
score=model.evaluate(x_test, y_test)

22235/22235 [==============================] - 113s 5ms/step


In [17]:
score[0]

1.6165685004964956

In [18]:
score[1]

0.043130200134922421

In [20]:
y_pred=model.predict(x_test)# for dataset of 2 characters
print (y_pred)
y_pred=numpy.argmax(y_pred, axis=1)
print (y_pred)

[[ 0.02702772  0.02702761  0.02702937 ...,  0.0270265   0.02702716
   0.02701131]
 [ 0.02702774  0.02702763  0.02702942 ...,  0.02702652  0.02702705
   0.02701133]
 [ 0.02702777  0.02702764  0.02702945 ...,  0.02702654  0.02702708
   0.0270113 ]
 ..., 
 [ 0.02702778  0.02702762  0.02702943 ...,  0.02702654  0.0270271
   0.02701128]
 [ 0.02702782  0.02702762  0.02702936 ...,  0.02702655  0.02702713
   0.02701133]
 [ 0.0270278   0.02702769  0.02702937 ...,  0.02702658  0.02702709
   0.02701137]]
[26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26
 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26]


In [21]:
y_pred=model.predict(x_test, batch_size=128, verbose=0)# for dataset of 2 characters
print (y_pred)
y_pred=numpy.argmax(y_pred, axis=1)
print (y_pred)

[[ 0.02702699  0.02702716  0.02702824 ...,  0.0270266   0.02702733
   0.0270237 ]
 [ 0.02702705  0.02702716  0.0270283  ...,  0.0270266   0.02702722
   0.02702371]
 [ 0.0270271   0.02702715  0.02702834 ...,  0.02702663  0.02702729
   0.0270237 ]
 ..., 
 [ 0.02702711  0.02702713  0.02702834 ...,  0.02702664  0.02702732
   0.02702369]
 [ 0.02702715  0.02702722  0.02702823 ...,  0.02702667  0.02702729
   0.02702368]
 [ 0.02702711  0.02702728  0.02702825 ...,  0.02702668  0.02702725
   0.02702371]]
[26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26
 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26]


In [37]:
#model=Sequential()
#y_pred=model.predict_classes(x_test)
#print(y_pred)

In [21]:
target=y_test
#print (classification_report(numpy.argmax(y_test, axis=1), y_pred, target))
print(confusion_matrix(numpy.argmax(y_test, axis=1),y_pred))

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [22]:
import pandas

pandas.DataFrame(hist.history).to_csv("D:Dataset_Final(March)/Checkpoints/Level_1/Epoch_60/Figure/History.csv")        
# visualizing losses and accuracy

train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['acc']
val_acc=hist.history['val_acc']
xc=range(epochs)

In [23]:
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.backends.backend_pdf import PdfPages

pp=PdfPages("D:Dataset_Final(March)/Checkpoints/Level_1/Epoch_60/Figure/Loss.pdf")
plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
print (plt.style.available)# use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])
plt.savefig(pp, format='pdf')
pp.close()

['bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark-palette', 'seaborn-dark', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'seaborn', 'Solarize_Light2', '_classic_test']


In [24]:
pp=PdfPages("D:Dataset_Final(March)/Checkpoints/Level_1/Epoch_60/Figure/Accuracy.pdf")
plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])
plt.savefig(pp, format='pdf')
pp.close()

In [29]:
# y_true=numpy.array(['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19',
#                    '20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36'])
# y_pred=numpy.array('26','26','26','26','26','26','26','26','26','26','26','26','26','26','26','26','26','26','26',
#                    '26','26','26','26','26','26','26','26','26','26','26','26','26','26','26','26','26')
y_true=['1','2','3']
y_pred=['2','2','2']
precision_recall_fscore_support(y_true, y_pred)

C:\Users\aliafridi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([ 0.        ,  0.33333333,  0.        ]),
 array([ 0.,  1.,  0.]),
 array([ 0. ,  0.5,  0. ]),
 array([1, 1, 1], dtype=int64))

In [30]:
print (f1_score(y_true, y_pred, average='macro'))
print (precision_score(y_true, y_pred, average='macro'))
print (recall_score(y_true, y_pred, average='macro'))

0.166666666667
0.111111111111
0.333333333333


C:\Users\aliafridi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\aliafridi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [26]:
#plt.imshow(x_train[0])